In [1]:
#Import earth engine
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=HqZzok-r2kNwrG9nye09U61r5DAb8_8y92KFzS5rgaA&tc=S9hvIyOSfKwntfSKJ1v0pFzKEovTESD9VZf2yF_ME8o&cc=amBzei-X3MuzhUaHKbisR9V6KSJwM90zwqpvRwdwh4g

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhXsQr8WZXiR9dveYN3f2-BbZKsctvdly3jQj_qd0VH-m2P1I3vq4NM

Successfully saved authorization token.


In [2]:
#Load countries data
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
USA = countries.filter(ee.Filter.eq('country_na', 'United States'))

In [3]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
.filterDate('2016-01-01', '2016-12-31').filterBounds(USA)\
.filter(ee.Filter.calendarRange(5, 9, 'month'))\
.sort('DATE_ACQUIRED')

# Get the median over time, in each band, in each pixel.
median = l8.median()

visParams = {'bands': ['B4', 'B3', 'B2'], 'max': 0.3}

In [5]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[40, -101], zoom_start = 4)

# Display the median composite.
my_map.add_ee_layer(median, visParams, 'median')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)